In [1]:
import pandas as pd

# CSV 파일 불러오기
master_info_df = pd.read_csv('seoul_bicycle_master_preprocessed.csv', encoding='utf-8')

# 데이터프레임 정보 확인
master_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3261 entries, 0 to 3260
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   stn_id    3261 non-null   object 
 1   stn_addr  3261 non-null   object 
 2   stn_lat   3261 non-null   float64
 3   stn_lng   3261 non-null   float64
dtypes: float64(2), object(2)
memory usage: 102.0+ KB


In [2]:
master_info_df.head()

,stn_id,stn_addr,stn_lat,stn_lng
0,ST-997,서울특별시 양천구 목동중앙로 49,37.534390,126.869598
1,ST-996,서울특별시 양천구 남부순환로88길5-16,37.524334,126.850548
2,ST-995,서울특별시 양천구 중앙로 153 공중화장실,37.510597,126.857323
3,ST-994,서울특별시 양천구 목동서로161,37.529163,126.872749
4,ST-993,서울특별시 양천구 신월로 342-1 구두수선대19,37.521511,126.857384


## 가까운 정류소 id 추가하기

In [3]:
# pip install haversine

In [4]:
from scipy.spatial import KDTree
from haversine import haversine

In [5]:
# 'stn_lat'과 'stn_lng'를 이용해 2차원 공간의 점을 표현하는 리스트를 만듭니다.
coords = master_info_df[['stn_lat', 'stn_lng']].values

# KDTree 객체를 생성합니다.
tree = KDTree(coords)

# 각 점에서 가장 가까운 이웃의 인덱스와 그 거리를 구합니다.
_, indices = tree.query(coords, k=2)  # k=2로 설정하면 자기 자신과 가장 가까운 이웃을 찾습니다.

# 가장 가까운 이웃이 자기 자신이므로, 두 번째로 가까운 이웃의 인덱스를 사용합니다.
nearby_indices = indices[:, 1]  # 두 번째로 가까운 이웃의 인덱스입니다.

# nearby_id 컬럼을 추가합니다.
master_info_df['nearby_id'] = master_info_df.loc[nearby_indices, 'stn_id'].values

# nearby_km 컬럼을 추가합니다.
master_info_df['nearby_km'] = [haversine(coords[i], coords[nearby_indices[i]]) for i in range(len(coords))]

In [6]:
master_info_df.head()

,stn_id,stn_addr,stn_lat,stn_lng,nearby_id,nearby_km
0,ST-997,서울특별시 양천구 목동중앙로 49,37.534390,126.869598,ST-310,0.219206
1,ST-996,서울특별시 양천구 남부순환로88길5-16,37.524334,126.850548,ST-1008,0.218592
2,ST-995,서울특별시 양천구 중앙로 153 공중화장실,37.510597,126.857323,ST-1504,0.162262
3,ST-994,서울특별시 양천구 목동서로161,37.529163,126.872749,ST-427,0.092895
4,ST-993,서울특별시 양천구 신월로 342-1 구두수선대19,37.521511,126.857384,ST-2761,0.175407


In [7]:
master_info_df['stn_gu'] = master_info_df['stn_addr'].str.split().str[1]

In [8]:
master_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3261 entries, 0 to 3260
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   stn_id     3261 non-null   object 
 1   stn_addr   3261 non-null   object 
 2   stn_lat    3261 non-null   float64
 3   stn_lng    3261 non-null   float64
 4   nearby_id  3261 non-null   object 
 5   nearby_km  3261 non-null   float64
 6   stn_gu     3261 non-null   object 
dtypes: float64(3), object(4)
memory usage: 178.5+ KB


In [9]:
master_info_df.describe()

,stn_lat,stn_lng,nearby_km
count,3261.000000,3261.000000,3261.000000
mean,37.548239,126.990918,0.168290
std,0.052808,0.092007,0.135714
min,37.403549,126.798042,0.000000
25%,37.505608,126.915245,0.078958
50%,37.546898,127.002235,0.144335
75%,37.578468,127.062752,0.228924
max,37.692322,127.180756,4.391333


## CSV 파일에 저장

In [10]:
# CSV 파일로 저장
master_info_df.to_csv('master_info_with_nearby.csv', index=False, encoding='utf-8')